In [2]:
#写在前面，感谢林大佬的开源数据处理，其次感谢鱼老的lgb参数，我的工作其实就是把他们的代码整合一起了。
#作者- - - - Ambromse
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
#import xgboost as xgb
#import catboost as ctb

/Users/sn0wfree/.conda/envs/CodersWheel/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
%pwd

'/Users/sn0wfree/PycharmProjects/auto_ml/auto_ml/core'

In [4]:
#请看林大佬的开源base
path = '/Users/sn0wfree/PycharmProjects/auto_ml/auto_ml/data/'


train_sales_data = pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/Train/train_sales_data.csv')
train_search_data= pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/Train/train_search_data.csv')
train_user_reply_data= pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/Train/train_user_reply_data.csv')

test = pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/evaluation_public.csv')

# submit_example = pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/submit_example.csv')

# train_sales_data = pd.read_csv(path + 'train_sales_data.csv')
# train_search_data = pd.read_csv(path + 'train_search_data.csv')
# train_user_reply_data = pd.read_csv(path + 'train_user_reply_data.csv')

# test = pd.read_csv(path + '/evaluation_public.csv')

data = pd.concat([train_sales_data, test], ignore_index=True)
data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']

num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum']
cate_feat = ['bodyType', 'model', 'province']

/Users/sn0wfree/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [5]:
data.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0


In [6]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum']
cate_feat = ['bodyType', 'model', 'province']

for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

In [7]:
data.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0


In [8]:
data.nunique()

adcode               22
bodyType              4
id                 5281
model                60
province             22
regMonth             12
regYear               3
popularity         7239
carCommentVolum     571
newsReplyVolum     1159
label              2965
dtype: int64

In [9]:
data.dtypes

adcode                int64
bodyType           category
id                    int64
model              category
province           category
regMonth              int64
regYear               int64
popularity          float64
carCommentVolum     float64
newsReplyVolum      float64
label               float64
dtype: object

In [10]:
print(train_sales_data.shape)
print(data.shape)

(31680, 7)
(36960, 11)


In [11]:
train = data[:31680]
y_label = train['label']
test = data[31680:]

drop_features = ['label']
train = train.drop(drop_features,axis=1)
test = test.drop(drop_features,axis=1)

In [12]:
train.shape
print(type(y_label))
train.head()

<class 'pandas.core.series.Series'>


,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0


In [13]:
#from sklearn.model_selection import KFold,StratifiedKFold

In [14]:
#此处请看鱼老知乎

#开始训练
import time
since = time.time()

x_score = []
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import f1_score
random_seed = 2018
#lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=395, max_depth=-1, learning_rate=0.1, n_estimators=5000, max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,min_child_weight=5, min_child_samples=5, subsample=0.8, subsample_freq=1,colsample_bytree=0.8, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True)
lgb_clf = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.01, n_estimators=5000,
                             max_bin=425, subsample_for_bin=50000, objective='regression_l1', min_split_gain=0,min_child_weight=5,
                             min_child_samples=10, subsample=0.8, subsample_freq=1,colsample_bytree=0.8,
                             reg_alpha=3, reg_lambda=0.1, seed=1000, n_jobs=-1, silent=True)

skf = StratifiedKFold(n_splits=5, random_state=random_seed, shuffle=True)
baseloss = []
loss = 0

res_prob = pd.DataFrame()
res = pd.DataFrame()
for i, (train_index, test_index) in enumerate(skf.split(train,y_label)):
    print("Fold", i)
    x_valid,y_valid = train.loc[test_index],y_label.loc[test_index]
    lgb_model = lgb_clf.fit(train.loc[train_index],y_label.loc[train_index],
                            eval_names=['train', 'valid'],
                            eval_metric='mae',                              
                            eval_set=[(train.loc[train_index], y_label.loc[train_index]),   
                                      (train.loc[test_index], y_label.loc[test_index])],early_stopping_rounds=100,verbose = 250)
    x_pred = lgb_model.predict(x_valid, num_iteration=lgb_model.best_iteration_)
    test_pred = lgb_model.predict(test, num_iteration=lgb_model.best_iteration_)
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss / 5)

time_elapsed = time.time() - since
print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # 打印出来时间

/Users/sn0wfree/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold 0
Training until validation scores don't improve for 100 rounds.
[250]	train's l1: 153.536	train's l1: 153.536	valid's l1: 465.061	valid's l1: 465.061
[500]	train's l1: 122.444	train's l1: 122.444	valid's l1: 399.582	valid's l1: 399.582
[750]	train's l1: 108.276	train's l1: 108.276	valid's l1: 371.741	valid's l1: 371.741
[1000]	train's l1: 99.263	train's l1: 99.263	valid's l1: 355.094	valid's l1: 355.094
[1250]	train's l1: 92.5136	train's l1: 92.5136	valid's l1: 342.712	valid's l1: 342.712
[1500]	train's l1: 87.34	train's l1: 87.34	valid's l1: 332.973	valid's l1: 332.973
[1750]	train's l1: 83.3577	train's l1: 83.3577	valid's l1: 325.74	valid's l1: 325.74
[2000]	train's l1: 79.8563	train's l1: 79.8563	valid's l1: 319.15	valid's l1: 319.15
[2250]	train's l1: 76.948	train's l1: 76.948	valid's l1: 313.961	valid's l1: 313.961
[2500]	train's l1: 74.4078	train's l1: 74.4078	valid's l1: 308.937	valid's l1: 308.937
[2750]	train's l1: 72.2157	train's l1: 72.2157	valid's l1: 304.919	valid's 

[1500]	train's l1: 112.675	train's l1: 112.675	valid's l1: 92.4979	valid's l1: 92.4979
[1750]	train's l1: 106.414	train's l1: 106.414	valid's l1: 89.6504	valid's l1: 89.6504
[2000]	train's l1: 101.567	train's l1: 101.567	valid's l1: 87.3066	valid's l1: 87.3066
[2250]	train's l1: 97.1422	train's l1: 97.1422	valid's l1: 85.3168	valid's l1: 85.3168
[2500]	train's l1: 93.4957	train's l1: 93.4957	valid's l1: 83.6368	valid's l1: 83.6368
[2750]	train's l1: 90.5173	train's l1: 90.5173	valid's l1: 82.3242	valid's l1: 82.3242
[3000]	train's l1: 87.7215	train's l1: 87.7215	valid's l1: 81.0687	valid's l1: 81.0687
[3250]	train's l1: 85.4025	train's l1: 85.4025	valid's l1: 80.047	valid's l1: 80.047
[3500]	train's l1: 83.3711	train's l1: 83.3711	valid's l1: 79.1669	valid's l1: 79.1669
[3750]	train's l1: 81.6142	train's l1: 81.6142	valid's l1: 78.4267	valid's l1: 78.4267
[4000]	train's l1: 80.1328	train's l1: 80.1328	valid's l1: 77.7481	valid's l1: 77.7481
[4250]	train's l1: 78.6785	train's l1: 78.678

In [20]:
testsub =pd.read_csv('/Users/sn0wfree/PycharmProjects/auto_ml/data/evaluation_public.csv')


testsub.head()
sum_proba = res.sum(axis=1) / 5.0
testsub['forecastVolum'] = sum_proba
testsub['forecastVolum'] = testsub[['forecastVolum']].astype(int)
testsub['forecastVolum'][testsub['forecastVolum']<0] =  0
testsub['forecastVolum'] = testsub['forecastVolum'].abs()
testsub[['id','forecastVolum']].to_csv('testsub.csv',header=True,index=None)

/Users/sn0wfree/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
res

,prob_0,prob_1,prob_2,prob_3,prob_4
0,124.913522,124.433842,124.765451,134.310908,141.024343
1,173.427118,198.747042,179.391770,185.802430,191.342399
2,94.950145,92.539747,100.112674,81.854029,109.272046
3,164.729014,173.497433,191.645979,162.078489,192.156553
4,208.877718,202.488468,218.151653,201.104082,217.455333
5,126.361120,124.508290,144.660291,139.908526,158.419040
6,279.486388,319.097950,351.834195,355.251137,318.757965
7,120.118849,146.297485,161.306804,139.319473,152.328550
8,434.211857,693.678286,883.543196,836.081216,780.806698
9,186.630428,236.153383,227.995235,224.815871,221.082247
